### 7. Комплексные запросы

Django ORM предоставляет возможности для создания сложных запросов с использованием аннотаций, выражений, а также подзапросов. Эти функции помогают гибко манипулировать данными, выполнять вычисления на уровне базы данных и оптимизировать работу с запросами.

#### 1. Аннотации и выражения (`annotate()`, `F()`, `Case()`, `When()`)

**Аннотации** позволяют добавлять вычисляемые поля к каждому объекту в наборе запросов, а **выражения** (expressions) помогают создавать вычисления и условия в запросах на уровне базы данных.

##### `annotate()`

Метод **`annotate()`** позволяет добавлять новое вычисляемое поле к каждому объекту QuerySet, основанное на агрегатных или других функциях. Это полезно для выполнения таких операций, как подсчёт связанных объектов или вычисление динамических значений.

Пример:

In [ ]:
from django.db.models import Count

# Подсчитать количество книг для каждого автора
authors = Author.objects.annotate(book_count=Count('book'))

for author in authors:
    print(author.name, author.book_count)

Здесь каждому объекту **Author** будет добавлено поле `book_count`, содержащее количество связанных книг.

##### `F()`

**`F()`** выражения позволяют ссылаться на значения других полей внутри того же объекта для выполнения операций с ними. Это полезно для операций на уровне базы данных без необходимости загружать объекты в память.

Пример:

In [ ]:
from django.db.models import F

# Увеличить цену каждой книги на 10%
Book.objects.update(price=F('price') * 1.10)

Этот запрос выполнит операцию умножения для всех записей напрямую в базе данных, без загрузки объектов в память.

##### `Case()` и `When()`

**`Case()`** и **`When()`** позволяют создавать условные выражения в запросах. Это похоже на конструкцию `if-else` в Python, но на уровне базы данных.

Пример:

In [ ]:
from django.db.models import Case, When, F

# Присвоить скидку в 20% для книг, цена которых больше 500, и 10% для остальных
books = Book.objects.annotate(
    discount=Case(
        When(price__gt=500, then=F('price') * 0.80),
        default=F('price') * 0.90
    )
)

for book in books:
    print(book.title, book.discount)

Здесь **`Case()`** позволяет выбрать разное поведение в зависимости от условия **`When()`**.

#### 2. Подзапросы (`Subquery`)

Подзапросы позволяют выполнять вложенные запросы, которые возвращают значения для использования в основном запросе. Django поддерживает подзапросы через класс **`Subquery`** и выражения, такие как **`OuterRef()`** для ссылок на внешние запросы.


##### `Subquery`

**`Subquery`** используется для вставки результатов одного запроса в другой. Например, можно выполнить подзапрос для получения последних значений из другой таблицы.

Пример:

In [ ]:
from django.db.models import Subquery, OuterRef

# Получить последнюю цену книги из таблицы History
latest_price = History.objects.filter(book=OuterRef('pk')).order_by('-date').values('price')[:1]

# Аннотировать каждую книгу последней ценой
books = Book.objects.annotate(latest_price=Subquery(latest_price))

for book in books:
    print(book.title, book.latest_price)

В этом примере подзапрос извлекает последнюю цену для каждой книги из таблицы **History**.

##### Пример использования `Subquery` с аннотацией

Можно использовать **`Subquery`** для сложных аннотаций и вычислений. Например, можно аннотировать модель значением, основанным на подзапросе:

In [ ]:
from django.db.models import Subquery, Count

# Найти количество заказов для каждого пользователя с помощью подзапроса
order_count = Order.objects.filter(user=OuterRef('pk')).values('user').annotate(count=Count('id')).values('count')

users = User.objects.annotate(total_orders=Subquery(order_count))

for user in users:
    print(user.username, user.total_orders)

В этом примере для каждого пользователя подсчитывается количество заказов с использованием подзапроса. **`OuterRef('pk')`** используется для ссылки на основной запрос.

#### Практическое применение

- **Аннотации** полезны для подсчёта связанных объектов, вычисления динамических полей или выполнения математических операций.
- **Выражения** позволяют делать более сложные вычисления и манипуляции с полями внутри запросов.
- **Подзапросы** необходимы, когда нужно встроить результат одного запроса в другой, например для сложных фильтров или аннотаций.

Используя эти возможности, студенты смогут создавать эффективные и гибкие запросы к базе данных, выполнять сложные вычисления на стороне сервера и оптимизировать свою работу с данными.